In [28]:
import os
import numpy as np
import torchvision
import torch
import cv2
from tqdm import tqdm

In [29]:
# from io import StringIO, BytesIO, TextIOWrapper
from zipfile import ZipFile 

# dataset = 'kagglewolvessanddogs_5340.zip'
# zipfile = ZipFile(BytesIO(dataset.read()))
with ZipFile('archive.zip','r') as zip_ref:
    zip_ref.extractall('UpdatedDataset')


In [51]:
REBUILD_DATA = True

deletable_file = []

class DogsVsWolves():
    IMG_SIZE = 60
    Dogs = "UpdatedDataset/data/dogs"
    Wolves = "UpdatedDataset/data/wolves"
    LABELS = {Wolves: 0, Dogs: 1}

    training_data =[]
    Dog_count = 0
    Wolves_count = 0

    
    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:    
                    path = os.path.join(label,f)
                    # print(path)

                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

                    try:
                        img = cv2.resize(img,(self.IMG_SIZE, self.IMG_SIZE))
                    except cv2.error as e:
                        # print("Configure error")
                        continue

                    label_encoded = np.eye(2)[self.LABELS[label]]
                    
                    # Ensure img is a 2D array
                    img = np.array(img)
                    
                    
                    try:
                        self.training_data.append([img, label_encoded])
                        # print(self.training_data)
                    except cv2.error as e:
                        pass

                    if label == self.Wolves:
                        self.Wolves_count += 1
                    elif label == self.Dogs:
                        self.Dog_count += 1

                except Exception as e:
                    # print(f"Error loading image: {path} - {str(e)}")
                    pass

        np.random.shuffle(self.training_data)
        # print(self.training_data)
        np.save("training_data.npy", self.training_data)
        print("Wolves", self.Wolves_count)
        print("Dogs", self.Dog_count)


if REBUILD_DATA:
    DogsVsWolves = DogsVsWolves()
    DogsVsWolves.make_training_data()


UpdatedDataset/data/wolves


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:05<00:00, 185.55it/s]


UpdatedDataset/data/dogs


100%|██████████| 1000/1000 [00:01<00:00, 683.14it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (2000, 2) + inhomogeneous part.

In [52]:
training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

EOFError: No data left in file